## Gesture Recognition Assignmnet

### Problem Statement.

 A home electronics goods manufacturer want to build a smart TV capable of recognising five types
of hand gestures which controls basic functioning of the TV without using remote. As a data scientist you are expected to develop this cool feature.

### Hand Guestures and expected functionality


- Thumbs up: Increase the volume

-  Thumbs down: Decrease the volume

-  Left swipe: 'Jump' backwards 10 seconds

-  Right swipe: 'Jump' forward 10 seconds

-  Stop: Pause the movie

### Data

 -  Data consists videos(a sequance of 30 images) split into train and test parts
   The training data consists of a few hundred videos categorized into one   
   of the five classes. Each of these videos have been recorded by various  
   people performing one of the five hand gestures in front of a webcam.

## Task
- our task is to build and train an appropriate CNN/RNN models on the 'train' data which performs well on the 'validation' data as well.

## Work Flow
1. Getting the data into storage
2. Creating data generators
3. Building a training pipeline
4. Cropping,resizing and normalisation of images
5. Building Conv3D Model
6. Building CNN+RNN Model



-

In [1]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip gdrive/My\ Drive/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

### Importing Libraries

In [3]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
import os
import imageio
import cv2# for 'imread' function
from skimage.transform import resize# for 'resize' function

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
import keras as Keras
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 51

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
#generator
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(source_path)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    # Cropped image of above dimension 
                    # (It will not change orginal image) 
                    
                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] /= 255
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if((len(source_path)%batch_size)//2==0):
            batch_size = 2
        else:
            batch_size = 1
        num_batches = len(source_path)%batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,18,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    
                    
                    # Cropped image of above dimension 
                    # (It will not change orginal image) 
                    
                    #image = image.crop((0, 0, 120, 120))
                    image = image.resize(100, 100)
                    
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] /= 255
                    batch_data[folder,idx,:,:,1] /= 255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] /= 255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 15 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


##  Model 1: Conv3D
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
Input_shape = (18, 100, 100, 3)
model = Sequential()
model.add(Conv3D(32, (3,3,3), padding='same',
                 input_shape=Input_shape))
#model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(32, (3, 3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))

model.add(Conv3D(64, (3, 3,3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv3D(64, (3, 3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 18, 100, 100, 32)  2624      
_________________________________________________________________
activation (Activation)      (None, 18, 100, 100, 32)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 100, 100, 32)  128       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 16, 98, 98, 32)    27680     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 98, 98, 32)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 98, 98, 32)    128       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 49, 49, 32)     0

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

### Model Compilation

In [13]:
optimiser = Keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'Gesture_recog' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

### Model Fitting

In [14]:
#ushus
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Source path =  Project_data/train ; batch size = 51
Epoch 1/15
13/13 [==============================] - 49s 145ms/step - loss: 1.6091 - categorical_accuracy: 0.2981 - val_loss: 1.6107 - val_categorical_accuracy: 0.0000e+00

Epoch 00001: saving model to Gesture_recog_2021-07-1114_08_23.506733/model-00001-1.60926-0.30769-1.61068-0.00000.h5
Epoch 2/15
13/13 [==============================] - 2s 186ms/step - loss: 1.6100 - categorical_accuracy: 0.1723 - val_loss: 1.6146 - val_categorical_accuracy: 0.0000e+00

Epoch 00002: saving model to Gesture_recog_2021-07-1114_08_23.506733/model-00002-1.60992-0.19048-1.61456-0.00000.h5
Epoch 3/15
13/13 [==============================] - 2s 164ms/step - loss: 1.6110 - categorical_accuracy: 0.1365 - val_loss: 1.6095 - val_categorical_accuracy: 0.0000e+00

Epoch 00003: saving model to Gesture_recog_2021-07-1114_08_23.506733/model-00003-1.61078-0.11538-1.60948-0.00000.h5
Epoch 4/15
13/13 [==============================] - 2s 152ms/step - loss: 1.6104 - cat

## Model 2 : CNN+RNN (Conv2D, LSTM)


In [15]:
#write your model here
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, LSTM, Bidirectional


Input_shape_1 = (18, 100, 100, 3)
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=Input_shape_1))
model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
 
model.add(TimeDistributed(Flatten()))
 
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=False, dropout=0.5))
model.add(Dense(5, activation='softmax'))
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 50, 50, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 48, 48, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 24, 24, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 24, 24, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 24, 24, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 12, 12, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 12, 12, 128)  

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

## Model compilation

In [16]:
#optimiser = Keras.optimizers.Adam(lr=0.001)
optimiser = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 18, 50, 50, 32)    4736      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 18, 48, 48, 32)    9248      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 18, 24, 24, 32)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 18, 24, 24, 64)    18496     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 18, 24, 24, 64)    36928     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 18, 12, 12, 64)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 18, 12, 12, 128)  

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [17]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [19]:
model_name = 'Gesture_recog' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 10,
  verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,
  min_lr = 0)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [20]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

## Model Fitting

In [21]:
#model_fit generator
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Source path =  Project_data/train ; batch size = 51
Epoch 1/15
13/13 [==============================] - 16s 260ms/step - loss: 1.6142 - categorical_accuracy: 0.4039 - val_loss: 1.5998 - val_categorical_accuracy: 0.0000e+00

Epoch 00001: saving model to Gesture_recog_2021-07-1114_08_23.506733/model-00001-1.61351-0.23077-1.59977-0.00000.h5
Epoch 2/15
13/13 [==============================] - 2s 183ms/step - loss: 1.5657 - categorical_accuracy: 0.4147 - val_loss: 1.5758 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to Gesture_recog_2021-07-1114_08_23.506733/model-00002-1.59662-0.28571-1.57577-0.50000.h5
Epoch 3/15
13/13 [==============================] - 2s 149ms/step - loss: 1.6402 - categorical_accuracy: 0.0536 - val_loss: 1.6212 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to Gesture_recog_2021-07-1114_08_23.506733/model-00003-1.63201-0.07692-1.62125-0.50000.h5
Epoch 4/15
13/13 [==============================] - 2s 164ms/step - loss: 1.6232 - categorical

In [ ]:
#h5 file Creation

In [22]:
from keras.datasets import cifar10
import numpy as np
import h5py

In [24]:
#Now mock up some simple dummy data to save to our file.

d1 = np.random.random(size = (1000,20))
d2 = np.random.random(size = (1000,200))

In [27]:
print(d1.shape) 
print(d2.shape)

(1000, 20)
(1000, 200)


In [28]:
hf = h5py.File('data.h5', 'w')

In [29]:
hf.create_dataset('Project_data/train', data=d1)
hf.create_dataset('Project_data/val', data=d2)

<HDF5 dataset "val": shape (1000, 200), type "<f8">

In [30]:
hf.close()

In [31]:
hf = h5py.File('data.h5', 'r')

In [32]:
hf.keys()

<KeysViewHDF5 ['Project_data']>

In [35]:
n1 = hf.get('Project_data/train')
n1

<HDF5 dataset "train": shape (1000, 20), type "<f8">


<HDF5 dataset "train": shape (1000, 20), type "<f8">

In [36]:
n1 = np.array(n1)

In [37]:
hf.close()

In [38]:
d1 = np.random.random(size = (100,33))
d2 = np.random.random(size = (100,333))
d3 = np.random.random(size = (100,3333))

In [39]:
hf = h5py.File('data.h5', 'w')

In [40]:
g1 = hf.create_group('group1')

In [41]:
g1.create_dataset('data1',data=d1)
g1.create_dataset('data2',data=d1)

<HDF5 dataset "data2": shape (100, 33), type "<f8">

In [42]:
g2 = hf.create_group('group2/subfolder')

In [43]:
g2.create_dataset('data3',data=d3)

<HDF5 dataset "data3": shape (100, 3333), type "<f8">

In [44]:
group2 = hf.get('group2/subfolder')

In [45]:
group2.items()

ItemsViewHDF5(<HDF5 group "/group2/subfolder" (1 members)>)

In [46]:
group1 = hf.get('group1')
group1.items()

ItemsViewHDF5(<HDF5 group "/group1" (2 members)>)

In [47]:
n1 = group1.get('data1')
np.array(n1).shape

(100, 33)

In [48]:
hf.close()

In [49]:
hf = h5py.File('data.h5', 'w')

hf.create_dataset('dataset_1', data=d1, compression="gzip", compression_opts=9)
hf.create_dataset('dataset_2', data=d2, compression="gzip", compression_opts=9)

hf.close()